In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px 


In [2]:
import os
os.getcwd()

'e:\\DATA SCIENCE\\timeseries_hackathon\\timeseries_data\\dashboard\\src'

In [3]:
import pandas as pd 



# // TODO : Use decimal formating 

DATA_PATH = 'pnbp_pernikahan.csv'


def process_data(data=DATA_PATH) : 
    data = pd.read_csv(DATA_PATH)  
    data.loc[:,'Tanggal Penerimaan'] = pd.to_datetime(data.loc[:,'Tanggal Penerimaan'])
    data= data.groupby('Tanggal Penerimaan').agg({'Nilai' : 'sum'})
    data.reset_index(inplace=True)
    return data 



DATA = process_data()

In [4]:
DATA['ts_log'] = DATA['Nilai'].apply(lambda x : np.log1p(x))

In [5]:
train_ratio = 0.7 

train_data = DATA.loc[:int(len(DATA)*train_ratio),'ts_log']
test_data = DATA.loc[int(len(DATA)*train_ratio):,'ts_log']

In [6]:
N_test_observations = len(test_data)
model_predictions = []

In [7]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [12]:
for time_point in range(N_test_observations):
    model = ARIMA(train_data, order=(1,0,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    
MAE_score = mean_absolute_error(test_data,model_predictions)
MAPE_score = mean_absolute_percentage_error(test_data,model_predictions)
RMSE_score = mean_squared_error(test_data,model_predictions)

C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\arima_model.py:472: F

ValueError: Found input variables with inconsistent numbers of samples: [376, 752]

In [13]:
model = ARIMA(train_data, order=(1,0,1))
model_fit = model.fit(disp=0)

C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


In [14]:
output = model_fit.forecast()

In [15]:
output

(array([14.10906842]),
 array([1.98651106]),
 array([[10.21557828, 18.00255857]]))

In [9]:
MAE_score

1.9995926144309264

In [10]:
MAPE_score

0.14062058701809466

In [11]:
RMSE_score

4.606694401648467

In [31]:
from sklearn.model_selection import TimeSeriesSplit
ts_cv = TimeSeriesSplit(n_splits=5)
def cross_validate_ts_arima(df,ts,p,d,q,cv=ts_cv) : 
    overall_MAE = list()
    overall_RMSE = list()
    overall_MAPE = list()
    for idx,(train_idx,val_idx) in enumerate(cv.split(df[ts])) : 
        train_data = df.loc[train_idx,ts]
        test_data = df.loc[val_idx,ts]
        len_test_data = len(test_data) 
        history_data = list()
        model_preds = list()
        for time_mark in range(len_test_data) : 
            model = ARIMA(train_data, order=(p, d, q))
            model_fit = model.fit(disp=0)
            pred_y = model_fit.forecast()[0]
            model_preds.append(pred_y)
            
        MAE_score = mean_absolute_error(test_data,model_preds)
        MAPE_score = mean_absolute_percentage_error(test_data,model_preds)
        RMSE_score = mean_squared_error(test_data,model_preds)
        # appending each loop MAE, RMSE , and MAPE to show average model performance 
        overall_MAE.append(MAE_score)
        overall_RMSE.append(RMSE_score)
        overall_MAPE.append(MAPE_score)
    print(f'Overall ARIMA Model with p d q = {p},{d} , {q} Performance on MAE Score = {np.mean(overall_MAE)}')
    print(f'Overall ARIMA Model with p d q = {p},{d} , {q} Performance on RMSE Score = {np.mean(overall_RMSE)}')
    print(f'Overall ARIMA Model with p d q = {p},{d} , {q} Performance on MAPE Score = {np.mean(overall_MAPE)}')
    # #append into dataframe 
    # model_result = pd.DataFrame({'Model_Name':  f'ARIMA Model with p={p} , d = {d} , q = {q} ', 
    #                             'AVG_MAE_SCORE' : {np.mean(overall_MAE)}, 
    #                             'AVG_RMSE_SCORE' :{np.mean(overall_RMSE)} , 
    #                             'AVG_MAPE_SCORE' : {np.mean(overall_MAPE)}})
    
    # model_result.to_csv(f'ARIMA_Model_Result_pdq=({p,d,q}).csv',index=False) 
    # model_result['Model_Name'] = f'ARIMA Model with p={p} , d = {d} , q = {q} '
    # model_result['AVG_MAE_SCORE'] = {np.mean(overall_MAE)}
    # model_result['AVG_RMSE_SCORE'] = {np.mean(overall_RMSE)}
    # model_result['AVG_MAPE_SCORE'] = {np.mean(overall_MAPE)}
    
    # model_result.to_csv(f'ARIMA_Model_Result_pdq=({p,d,q}).csv',index=False) 
    

In [32]:
cross_validate_ts_arima(DATA,'ts_log',1,0,1)

C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\arima_model.py:472: F

Overall ARIMA Model with p d q = 1,0 , 1 Performance on MAE Score = 1.7570360627673374
Overall ARIMA Model with p d q = 1,0 , 1 Performance on RMSE Score = 4.166927776353077
Overall ARIMA Model with p d q = 1,0 , 1 Performance on MAPE Score = 0.12955460106646888


In [45]:
from sklearn.model_selection import TimeSeriesSplit
ts_cv = TimeSeriesSplit(n_splits=5)
from statsmodels.tsa.statespace.sarimax import SARIMAX
def cross_validate_ts_SARIMA(df,ts,p,d,q,P,D,Q,s,cv=ts_cv) : 
    overall_MAE = list()
    overall_RMSE = list()
    overall_MAPE = list()
    for idx,(train_idx,val_idx) in enumerate(cv.split(df[ts])) : 
        predictions = np.array([])
        train_data = df.loc[train_idx,ts]
        test_data = df.loc[val_idx,ts]
        len_test_data = len(test_data) 
        # history_data = list()
        # model_preds = list()
        model = SARIMAX(train_data, order=(p, d, q),seasonal_order =(P,D,Q,s) )
        model_fit = model.fit(disp=0)
        forecast = model_fit.predict()
        predictions = np.concatenate(predictions, forecast)
        # model_preds.append(pred_y)
        print(prediction)
        MAE_score = mean_absolute_error(test_data,predictions)
        MAPE_score = mean_absolute_percentage_error(test_data,predictions)
        RMSE_score = mean_squared_error(test_data,predictions)
        # appending each loop MAE, RMSE , and MAPE to show average model performance 
        overall_MAE.append(MAE_score)
        overall_RMSE.append(RMSE_score)
        overall_MAPE.append(MAPE_score)
        
    print(f'Overall SARIMA Model with p d q = {p},{d} , {q} seasonal order = {P,D,Q,s} Performance on MAE Score = {np.mean(overall_MAE)}')
    print(f'Overall SARIMA Model with p d q = {p},{d} , {q} seasonal order = {P,D,Q,s} Performance on RMSE Score = {np.mean(overall_RMSE)}')
    print(f'Overall SARIMA Model with p d q = {p},{d} , {q} seasonal order = {P,D,Q,s} Performance on MAPE Score = {np.mean(overall_MAPE)}')
    
    
    
    

In [55]:
from sklearn.model_selection import TimeSeriesSplit
ts_cv = TimeSeriesSplit(n_splits=5)
def cross_validate_ts_SARIMA(df,ts,p,d,q,P,D,Q,s,cv=ts_cv) : 
    overall_MAE = list()
    overall_RMSE = list()
    overall_MAPE = list()
    for idx,(train_idx,val_idx) in enumerate(cv.split(df[ts])) : 
        train_data = df.loc[train_idx,ts]
        test_data = df.loc[val_idx,ts]
        len_test_data = len(test_data) 
        history_data = list()
        model_preds = list()
        for time_mark in range(len_test_data) : 
            model = SARIMAX(train_data, order=(p, d, q),seasonal_order=(P,D,Q,s))
            model_fit = model.fit(disp=0)
            pred_y = model_fit.predict()[0]
            model_preds.append(pred_y)
            
        MAE_score = mean_absolute_error(test_data,model_preds)
        MAPE_score = mean_absolute_percentage_error(test_data,model_preds)
        RMSE_score = mean_squared_error(test_data,model_preds)
        # appending each loop MAE, RMSE , and MAPE to show average model performance 
        overall_MAE.append(MAE_score)
        overall_RMSE.append(RMSE_score)
        overall_MAPE.append(MAPE_score)
    print(f'Overall SARIMAX Model with p d q = {p},{d} , {q}  Seasonal Order = {P,D,Q,s} Performance on MAE Score = {np.mean(overall_MAE)}')
    print(f'Overall SARIMAX Model with p d q = {p},{d} , {q} Seasonal Order = {P,D,Q,s} Performance on RMSE Score = {np.mean(overall_RMSE)}')
    print(f'Overall SARIMAX Model with p d q = {p},{d} , {q} Seasonal Order = {P,D,Q,s} Performance on MAPE Score = {np.mean(overall_MAPE)}')
    # #append into dataframe 
    # model_result = pd.DataFrame({'Model_Name':  f'ARIMA Model with p={p} , d = {d} , q = {q} ', 
    #                             'AVG_MAE_SCORE' : {np.mean(overall_MAE)}, 
    #                             'AVG_RMSE_SCORE' :{np.mean(overall_RMSE)} , 
    #                             'AVG_MAPE_SCORE' : {np.mean(overall_MAPE)}})
    
    # model_result.to_csv(f'ARIMA_Model_Result_pdq=({p,d,q}).csv',index=False) 
    # model_result['Model_Name'] = f'ARIMA Model with p={p} , d = {d} , q = {q} '
    # model_result['AVG_MAE_SCORE'] = {np.mean(overall_MAE)}
    # model_result['AVG_RMSE_SCORE'] = {np.mean(overall_RMSE)}
    # model_result['AVG_MAPE_SCORE'] = {np.mean(overall_MAPE)}
    
    # model_result.to_csv(f'ARIMA_Model_Result_pdq=({p,d,q}).csv',index=False) 
    

In [56]:
cross_validate_ts_SARIMA(
    
    df = DATA , ts = 'ts_log', 
    p=1, d=0,q=1 , 
    P=1,D=0,Q=1,s = 12,
    cv = ts_cv1
)

C:\Users\Fakhri Robi Aulia\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


Overall SARIMAX Model with p d q = 1,0 , 1  Seasonal Order = (1, 0, 1, 12) Performance on MAE Score = 15.01775659950501
Overall SARIMAX Model with p d q = 1,0 , 1 Seasonal Order = (1, 0, 1, 12) Performance on RMSE Score = 229.75484180050594
Overall SARIMAX Model with p d q = 1,0 , 1 Seasonal Order = (1, 0, 1, 12) Performance on MAPE Score = 1.0


In [53]:
model = SARIMAX(DATA['ts_log'] , p=1,d=0,q=1,P=1,D=0,Q=1,s=12)
model_fit = model.fit(disp=0)
pred_y = model_fit.predict()

In [54]:
pred_y

0        0.000000
1       12.159540
2       13.930660
3       14.773644
4       14.757545
          ...    
1246    13.136213
1247    13.182753
1248    16.033708
1249    16.176993
1250    11.339205
Name: predicted_mean, Length: 1251, dtype: float64

In [ ]:
# OLS Model 
